<a href="https://colab.research.google.com/github/JayThibs/hyperdrive-vs-automl-plus-deployment/blob/main/automl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Automated ML

# Environment and Import Dependencies

Here we specify the conda dependencies.

In [1]:
%%writefile conda_dependencies.yml

dependencies:
- python=3.6.2
- pip=20.2.4
- pip:
    - azureml-defaults
    - scikit-learn

Overwriting conda_dependencies.yml


In [2]:
from azureml.core import Environment

# Creating a conda environment for model training. It needs to be included in ScriptRunConfig.

sklearn_env = Environment.from_conda_specification(name='sklearn_env', file_path='./conda_dependencies.yml')

In [3]:
import pandas as pd
from pandas.api.types import is_string_dtype, is_numeric_dtype, is_categorical_dtype
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

import logging
import os
import csv

from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.24.0


In [4]:
import pandas as pd
from pandas.api.types import is_string_dtype, is_numeric_dtype, is_categorical_dtype
import numpy as np
import os
from sklearn.model_selection import train_test_split
import datetime

# Data Exploration was done in a notebook beforehand


def dates(X):
    """
    date_recorded: this might be a useful variable for this analysis, although the year itself would be 
    useless in a practical scenario moving into the future. We will convert this column into a datetime, 
    and we will also create 'year_recorded' and 'month_recorded' columns just in case those levels prove 
    to be useful. A visual inspection of both casts significant doubt on that possibility, but we'll proceed 
    for now. We will delete date_recorded itself, since random forest cannot accept datetime
    """
    for i in [X]:
        i['date_recorded'] = pd.to_datetime(i['date_recorded'])
        i['year_recorded'] = i['date_recorded'].apply(lambda x: x.year)
        i['month_recorded'] = i['date_recorded'].apply(lambda x: x.month)
        i['date_recorded'] = (pd.to_datetime(i['date_recorded'])).apply(lambda x: x.toordinal())
    return X


def date_parser(df):
    date_recorder = list(map(lambda x: datetime.datetime.strptime(str(x), '%Y-%m-%d'),
                             df['date_recorded'].values))
    df['year_recorder'] = list(map(lambda x: int(x.strftime('%Y')), date_recorder))
    df['weekday_recorder'] = list(map(lambda x: int(x.strftime('%w')), date_recorder))
    df['yearly_week_recorder'] = list(map(lambda x: int(x.strftime('%W')), date_recorder))
    df['month_recorder'] = list(map(lambda x: int(x.strftime('%m')), date_recorder))
    df['age'] = df['year_recorder'].values - df['construction_year'].values
    del df['date_recorded']
    return df


def bools(X):
    """
    public_meeting: we will fill the nulls as 'False'
    permit: we will fill the nulls as 'False'
    """
    z = ['public_meeting', 'permit']
    for i in z:
        X[i].fillna(False, inplace = True)
        X[i] = X[i].apply(lambda x: float(x))
    return X


def small_n(X):
    "Collapsing small categorical value counts into 'other'"
    cols = [i for i in X.columns if type(X[i].iloc[0]) == str]
    X[cols] = X[cols].where(X[cols].apply(lambda x: x.map(x.value_counts())) > 100, "other")
    return X


## Dataset

### Overview

We'll be using the Pump it Up dataset from the DrivenData competition.

The description of the problem: 

> Using data from Taarifa and the Tanzanian Ministry of Water, can you predict which pumps are functional, which need some repairs, and which don't work at all? This is an intermediate-level practice competition. Predict one of these three classes based on a number of variables about what kind of pump is operating, when it was installed, and how it is managed. A smart understanding of which waterpoints will fail can improve maintenance operations and ensure that clean, potable water is available to communities across Tanzania.

In other words, our goal is to predict which water pumps are non-functioning or functioning, but in need of repair.

In this project, we will train a model using AutoML to train multiple multiple and choose the best performing model for deployment.

In [17]:
# We loaded the dataset into Azure and we are grabbing it here.

from azureml.core import Workspace, Experiment

# download config file in azure and put it in the current Notebooks folder
ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="Pump-it-Up-Data-Mining-the-Water-Table")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

dataset = Dataset.get_by_name(ws, name='Pump-it-Up-dataset')
df = dataset.to_pandas_dataframe()
y = df['status_group']
del df['status_group']
X = df
del df

Workspace name: quick-starts-ws-142186
Azure region: southcentralus
Subscription id: f5091c60-1c3c-430f-8d81-d802f6bf2414
Resource group: aml-quickstarts-142186


In [18]:
X.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,...,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,...,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,...,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,...,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,...,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


In [19]:
# A lot of null values for construction year. Of course, this is a missing value (a placeholder).
# For modeling purposes, this is actually fine, but we'll have trouble with visualizations if we
# compare the results for different years, so we'll set the value to something closer to
# the other values that aren't placeholders. Let's look at the unique years and set the null
# values to 50 years sooner.
# Alright, let's set it to 1910
X.loc[X['construction_year'] < 1950, 'construction_year'] = 1910
X['population'] = np.log(X['population'])

# Cleaning up the features of our dataset
X = dates(X)
x = date_parser(X)
X = bools(X)
X['population'] = np.log(X['population'])
X = small_n(X)

# Splitting the dataset into a training and test set
# Test set will be used later
# The same random seed (42) for the Hyperdrive model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Concatenating the features and labels together to feed to our AutoML model
clean_df = pd.concat([X_train, y_train], axis=1)

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/pandas/core/series.py:856: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


ValueError: time data '734210' does not match format '%Y-%m-%d'

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Get the default datastore to be entered as a parameter in tabular dataset creation
datastore = workspace.get_default_datastore()

# Change pandas dataframe into a tabular dataset to be used in automl
training_data = TabularDatasetFactory.register_pandas_dataframe(clean_df, datastore, 'automl_data')

In [ ]:
training_data.take(5).to_pandas_dataframe()

# Setting up Experiment

We'll create a new experiment for our deployment of an AutoML model and create a project folder to hold the training scripts.

In [ ]:
experiment_name = 'automl-pump-it-up-operationalize'
project_folder = './automl-pipeline-project'

automl_experiment = Experiment(workspace, experiment_name)
automl_experiment

In [21]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Creating a compute cluster if there isn't one that is already created.

cpu_cluster_name = 'hypr-auto-clustr'

try:
    cpu_cluster = ComputeTarget(workspace=workspace, name=cpu_cluster_name)
    print('Found existing compute target.')
except ComputeTargetException:
    print('Creating a new computer target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_v2',
                                                          max_nodes=4)
    cpu_cluster = ComputeTarget.create(workspace, cpu_cluster_name, compute_config)
    
cpu_cluster.wait_for_completion(show_output=True)

NameError: name 'workspace' is not defined

# AutoML Configuration

We'll create a new experiment for our deployment of an AutoML model and create a project folder to hold the training scripts.

Here we create the general AutoML settings object.


Calculate recall to test how well we do on True Positives. We can imagine a real scenario where we want to build a model that does not miss the non-functioning water pumps, and we care much less functioning water pumps that are incorrectly predicted as non-functional. Recall is useful to make sure we miss less True Positives.

In [22]:
from azureml.train.automl import AutoMLConfig

automl_settings = {
    "experiment_timeout_minutes": 20, # to set a limit on the amount of time AutoML will be running
    "max_concurrent_iterations": 5, # applies to the compute target we are using
    "primary_metric" : 'norm_macro_recall' # recall for our performance metric
}

# Setting AutoML config for model training.

automl_config = AutoMLConfig(compute_target=cpu_cluster,
                             task = "classification", # classifying if water pumps are functional
                             training_data=training_data, 
                             label_column_name="status_group", # our target variable for water pump function  
                             path = project_folder,
                             enable_early_stopping= True, # prevents automl from spending too much time on models that stopped improving, saves time and compute costs
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

NameError: name 'cpu_cluster' is not defined

## Create Pipeline and AutoMLStep

Defining the outputs for the AutoMLStep using TrainingOutput.

In [23]:
from azureml.pipeline.core import PipelineData, TrainingOutput

ds = workspace.get_default_datastore()
metrics_output_name = 'metrics_output'
best_model_output_name = 'best_model_output'

metrics_data = PipelineData(name='metrics_data',
                           datastore=ds,
                           pipeline_output_name=metrics_output_name,
                           training_output=TrainingOutput(type='Metrics'))
model_data = PipelineData(name='model_data',
                           datastore=ds,
                           pipeline_output_name=best_model_output_name,
                           training_output=TrainingOutput(type='Model'))

NameError: name 'workspace' is not defined

## Create the AutoMLStep

In [24]:
# Creating an AutoMLStep

automl_step = AutoMLStep(
    name='automl_module',
    automl_config=automl_config,
    outputs=[metrics_data, model_data],
    allow_reuse=True
    )

NameError: name 'automl_config' is not defined

In [25]:
# Creating a Pipeline

from azureml.pipeline.core import Pipeline

pipeline = Pipeline(
    description="pipeline_with_automlstep",
    workspace=workspace,    
    steps=[automl_step])

NameError: name 'workspace' is not defined

In [26]:
print('Submitting AutoML experiment...')

pipeline_run = automl_experiment.submit(pipeline)

Submitting AutoML experiment...


NameError: name 'automl_experiment' is not defined

# Run Details

Using the RunDeatils widget to show the different experiments.

In [1]:
from azureml.widgets import RunDetails
RunDetails(pipeline_run).show()

NameError: name 'pipeline_run' is not defined

In [ ]:
pipeline_run.wait_for_completion()

# Examine Results

# Retrive the metrics of all child runs

In [ ]:
metrics_output = pipeline_run.get_pipeline_output(metrics_output_name)
num_file_downloaded = metrics_output.download('.', show_progress=True)

In [ ]:
import json
with open(metrics_output._path_on_datastore) as f:
    metrics_output_result = f.read()
    
deserialized_metrics_output = json.loads(metrics_output_result)
df = pd.DataFrame(deserialized_metrics_output)
df

# Best Model

In [ ]:
# Retrieve best model from Pipeline Run
best_model_output = pipeline_run.get_pipeline_output(best_model_output_name)
num_file_downloaded = best_model_output.download('.', show_progress=True)

NameError: name 'pipeline_run' is not defined

In [ ]:
import pickle

with open(best_model_output._path_on_datastore, "rb" ) as f:
    best_model = pickle.load(f)
best_model

In [ ]:
best_model.steps

NameError: name 'best_model' is not defined

# Test the model on the Test Set

In [ ]:
# Predict on the Test Set
ypred = best_model.predict(X_test)

# calculate recall
recall = recall_score(y_test, ypred, average='micro')
print('Recall: %.3f' % recall)

NameError: name 'best_model' is not defined

# Model Deployment

Registering the model, creating an inference config and deploy the model as a web service.

In other words, we are publishing the pipeline to enable a REST endpoint to rerun the pipeline from any HTTP library on any platform.

In [ ]:
published_pipeline = pipeline_run.publish_pipeline(
    name="Pump it Up Train", description="Training Pump it Up pipeline", version="1.0")

published_pipeline

NameError: name 'pipeline_run' is not defined

Now we authenticate to retrieve the auth_header so that the endpoint can be used.

In [ ]:
from azureml.core.authentication import InteractiveLoginAuthentication

interactive_auth = InteractiveLoginAuthentication()
auth_header = interactive_auth.get_authentication_header()

# Test the Deployed Model

Here we will send a request to the deployed model to test it.



In [ ]:
import requests

# Geting the REST url from the endpoint property of the published pipeline
rest_endpoint = published_pipeline.endpoint

# Building an HTTP POST request to the endpoint
# We also add a JSON payload object with the experiment name
response = requests.post(rest_endpoint, 
                         headers=auth_header, 
                         json={"ExperimentName": "pipeline-rest-endpoint"}
                        )

print(response.status_code)
print(response.elapsed)
print(response.json())

In [ ]:
headers = {'Content-Type':'application/json'}
data = {"text": ['the food was horrible', 
                 'wow, this movie was truely great, I totally enjoyed it!',
                 'why the heck was my package not delivered in time?']}

resp = requests.post(aci_service.scoring_uri, json=data, headers=headers)
print("Prediction Results:", resp.json())

We are making it so that a request will trigger the run. We are also going to access the Id key from the response dict to get the value of the run id.

In [ ]:
try:
    response.raise_for_status()
except Exception:    
    raise Exception("Received bad response from the endpoint: {}\n"
                    "Response Code: {}\n"
                    "Headers: {}\n"
                    "Content: {}".format(rest_endpoint, response.status_code, response.headers, response.content))

run_id = response.json().get('Id')
print('Submitted pipeline run: ', run_id)

# Printing the logs of the web service

We can now use the run id to monitor the status of the new run. 

In [ ]:
from azureml.pipeline.core.run import PipelineRun
from azureml.widgets import RunDetails

published_pipeline_run = PipelineRun(ws.experiments["pipeline-rest-endpoint"], run_id)
RunDetails(published_pipeline_run).show()

# Printing the logs and Deleting the Service

In [ ]:
# Delete computer target in order to avoid incurring additional charges.

AmlCompute.delete(cpu_cluster)